In [3]:
from google.colab import drive

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns


In [5]:
drive.mount('/content/gdrive')
DATADIR = '/content/gdrive/MyDrive/KaggleStoreSales/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
train = pd.read_csv(DATADIR + 'trainSupplementaryLabelEncoding.csv')
train = train[train.columns.difference(["id"])]
print(train.info())
y = train[['sales']]
x = train[train.columns.difference(["sales"])]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   city          float64
 1   cluster       int64  
 2   date          int64  
 3   dcoilwtico    float64
 4   family        float64
 5   holiday_type  float64
 6   onpromotion   int64  
 7   sales         float64
 8   state         float64
 9   store_nbr     int64  
 10  transactions  float64
 11  type          float64
dtypes: float64(8), int64(4)
memory usage: 274.7 MB
None


In [76]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor(n_estimators=10)
model.fit(x, y.values.ravel())
print(model.feature_importances_)

[0.02143401 0.07428689 0.02709667 0.44764854 0.00431817 0.26828963
 0.02384838 0.11009232 0.02298538]


In [7]:
x = x.drop('city', axis=1)
x = x.drop('state', axis=1)
x = x.drop('holiday_type', axis=1)
print(x.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   cluster       int64  
 1   date          int64  
 2   dcoilwtico    float64
 3   family        float64
 4   onpromotion   int64  
 5   store_nbr     int64  
 6   transactions  float64
 7   type          float64
dtypes: float64(4), int64(4)
memory usage: 183.2 MB
None


In [ ]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.1)
selector.fit(x)
x = x[x.columns[selector.get_support(indices=True)]]

In [77]:
print(x.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   cluster       int64  
 1   date          int64  
 2   dcoilwtico    float64
 3   family        float64
 4   holiday_type  float64
 5   onpromotion   int64  
 6   store_nbr     int64  
 7   transactions  float64
 8   type          float64
dtypes: float64(5), int64(4)
memory usage: 206.1 MB
None


In [ ]:
#PCA
from sklearn.decomposition import PCA
reducedNo = 0
pca = PCA(n_components=2)
for i in range (5,83):
    pca= PCA(n_components=i)
    principalComponents = pca.fit_transform(x)
    print ('for ' + str(i) + ' total information preserved: ' + str(sum(pca.explained_variance_ratio_)) )
    if (sum(pca.explained_variance_ratio_) == 100):
            reducedNo = i
            break
print(reducedNo)

In [ ]:
from sklearn.decomposition import PCA
xPCA = x
pca = PCA(n_components=8)
principalComponents = pca.fit_transform(xPCA)
x = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5','principal component 6','principal component 7','principal component 8'])
print ('for ' + str(8) + ' total information preserved: ' + str(sum(pca.explained_variance_ratio_)) )

for 8 total information preserved: 0.9999851845648784


In [8]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)
x = poly.fit_transform(x)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, shuffle=False)

In [64]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
xTrainScaled = scaler.fit_transform(x_train)
xValidScaled = scaler.transform(x_valid)
xTestScaled = scaler.transform(xTest)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
from sklearn.linear_model import LinearRegression
modelLG = LinearRegression()
modelLG.fit(x_train, y_train)
predictions = modelLG.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
#print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

In [ ]:
from sklearn import linear_model
reg = linear_model.BayesianRidge()
reg.fit(x_train, y_train.values.ravel())
predictions = reg.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
#print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

R2 Score: 0.20696409892911294
MSE Score: 1949938.379767444
MAPE Score: 3.9605190574498694e+17


In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=5, random_state=0,
    loss='squared_error'
).fit(x_train, y_train.values.ravel())
predictions = est.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
#print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

R2 Score: 0.7192796224809186
MSE Score: 690242.9478515293
MAPE Score: 3.0649830578705284e+16


In [75]:
from sklearn.linear_model import TweedieRegressor
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(),  GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=10, random_state=0,
    loss='squared_error'
))
pipe.fit(x_train, y_train.values.ravel())
predictions = pipe.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
#print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

R2 Score: 0.8092832523592141
MSE Score: 468939.55921416596
MAPE Score: 4.244579154077163e+16


In [59]:
from sklearn import tree
dt = tree.DecisionTreeRegressor()
dt.fit(x_train, y_train)
predictions = dt.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
#print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

R2 Score: 0.4280097390008263
MSE Score: 1406425.3097109015
MAPE Score: 3.2216990233285804e+16


In [ ]:
from sklearn import neighbors
knn = neighbors.KNeighborsRegressor(1, weights='distance', algorithm='kd_tree', leaf_size=60)
predictions = knn.fit(x_train,y_train).predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

R2 Score: 0.40347342070733405
MSE Score: 1466755.8807153963
MSLE Score: 1.9018107517529865
MAPE Score: 8505588771962589.0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(x, y.values.ravel())
predicitions = regr.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


R2 Score: 0.40119973119551033
MSE Score: 1472346.490720306
MSLE Score: 1.9373897004639395
MAPE Score: 8591700032907890.0


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
reg2 = GradientBoostingRegressor(
    n_estimators=, learning_rate=0.1, max_depth=10, random_state=0,
    loss='squared_error'
)
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg2), ('lr', reg3)])
ereg = ereg.fit(x_train, y_train.values.ravel())
predictions = ereg.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
#print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

R2 Score: 0.6910348953952858
MSE Score: 759691.8558971301
MAPE Score: 2.1729578041115533e+17


In [74]:
from sklearn.linear_model import TweedieRegressor
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(), TweedieRegressor(power=1, alpha=0.5, link='identity'))
pipe.fit(x_train, y_train.values.ravel())
predictions = pipe.predict(x_test)
print('R2 Score: '+str(r2_score(y_test, predictions)))
print('MSE Score: '+str(mean_squared_error(y_test, predictions)))
#print('MSLE Score: '+str(mean_squared_log_error(y_test, predictions)))
print('MAPE Score: '+str(mean_absolute_percentage_error(y_test, predictions)))

R2 Score: -0.03220004784744224
MSE Score: 2538001.7300321544
MAPE Score: 3.258905480850986e+17


In [ ]:
testData = pd.read_csv(DATADIR + 'testSupplementaryLabelEncoding.csv')
test = testData[testData.columns.difference(["id"])]
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          28512 non-null  float64
 1   cluster       28512 non-null  int64  
 2   date          28512 non-null  int64  
 3   dcoilwtico    28512 non-null  float64
 4   family        28512 non-null  float64
 5   holiday_type  28512 non-null  float64
 6   onpromotion   28512 non-null  int64  
 7   state         28512 non-null  float64
 8   store_nbr     28512 non-null  int64  
 9   transactions  28512 non-null  float64
 10  type          28512 non-null  float64
dtypes: float64(7), int64(4)
memory usage: 2.4 MB
None


In [ ]:
test = test.reindex(labels=train.columns,axis=1)
test = test.fillna(0)

In [ ]:
xTest = test[x.columns]
print(xTest.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cluster       28512 non-null  int64  
 1   date          28512 non-null  int64  
 2   dcoilwtico    28512 non-null  float64
 3   family        28512 non-null  float64
 4   holiday_type  28512 non-null  float64
 5   onpromotion   28512 non-null  int64  
 6   store_nbr     28512 non-null  int64  
 7   transactions  28512 non-null  float64
 8   type          28512 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 2.0 MB
None


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(xTest)
xTest = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5'])
print ('for ' + str(5) + ' total information preserved: ' + str(sum(pca.explained_variance_ratio_)) )

for 15 total information preserved: 0.9958188346187723


In [ ]:
print(xTest)

       principal component 1  ...  principal component 5
0                  -7.320975  ...              -0.631222
1                  -7.320977  ...              -0.631223
2                  -5.321214  ...              -0.629893
3                  12.677907  ...              -0.617155
4                  -7.320977  ...              -0.631223
...                      ...  ...                    ...
28507              -6.293106  ...              -0.837660
28508              -7.293187  ...              -0.838447
28509              -6.290853  ...              -0.836288
28510               1.705720  ...              -0.832476
28511              -7.293153  ...              -0.838426

[28512 rows x 5 columns]


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)

LinearRegression()

In [ ]:
predictions = model.predict(xTest)

In [ ]:
from sklearn.svm import LinearSVR

svm = LinearSVR(epsilon=1.5)
svm.fit(x,y)
predictions = svm.predict(xTest)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn import neighbors
knn = neighbors.KNeighborsRegressor(3, weights='distance')
predictions = knn.fit(x,y).predict(xTest)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.8, shuffle=False)



In [ ]:
scaler = StandardScaler()
xTrainScaled = scaler.fit_transform(x_train)
xValidScaled = scaler.transform(x_valid)
xTestScaled = scaler.transform(xTest)



In [ ]:
x_train

In [ ]:
from tensorflow import keras
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=1)
model = keras.models.Sequential([
    keras.layers.Dense(30,activation='linear', input_shape=x_train.shape[1:]),
    keras.layers.Dense(20,activation='linear', input_shape=x_train.shape[1:]),
    keras.layers.Dense(10,activation='linear', input_shape=x_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss='mean_squared_logarithmic_error', optimizer='sgd')
history = model.fit(xTrainScaled, y_train, epochs=3, validation_data=(xValidScaled, y_valid), callbacks=[callback])


Epoch 1/3
18756/18756 [==============================] - 186s 10ms/step - loss: 5.1081 - val_loss: 4.7357
Epoch 2/3
18756/18756 [==============================] - 196s 10ms/step - loss: 4.9264 - val_loss: 4.7879
Epoch 3/3
18756/18756 [==============================] - 197s 11ms/step - loss: 4.8852 - val_loss: 4.6292


In [ ]:
predictions = knn.predict(xTest)

In [ ]:
print(predictions)

[[  0.  ]
 [  0.  ]
 [229.  ]
 ...
 [ 60.99]
 [  0.  ]
 [  0.  ]]


In [ ]:
testData = pd.read_csv(DATADIR + 'testSupplementary.csv')
ids = testData['id'].values
#print(ids)
#print(prediction)
d = {'id':ids,'sales':predictions}
df = pd.DataFrame(d)

Exception: ignored

In [ ]:
testData = pd.read_csv(DATADIR + 'testSupplementary.csv')
ids = testData['id'].values
#print(ids)
prediction = []
for row in predictions:
    for elem in row:
        prediction.append(elem)
#print(prediction)
d = {'id':ids,'sales':prediction}
df = pd.DataFrame(d)

In [ ]:
df.to_csv(DATADIR + 'submission.csv', index=False)